# Read me
Dynamic Spectra (DySpec) is a spectrum-transforming fluorescent barcode coupled with time-lapse imaging to exponentially increase the number of co-imaged proteins. Theoretically, if *F* fluorophores are imaged in each cycle, *N* cycles of image can visualize *F<sup>N</sup>* proteins. To decode proteins from DySpec images, we developed two customized algorithms here: `Image subtraction` and `Linear unmixing`. The step-by-step operations and critical considerations are outlined in detail below.

### Hardware
- Computer workstation equipped with an AMD Ryzen 5975WX CPU
- NVIDIA RTX 3090 graphics processing card

### Software
- [Python](https://www.python.org/).
- [Jupyter Notebook](https://jupyter.org/).
- [Fiji](https://imagej.net/software/fiji/downloads)
- [ZEN](https://www.zeiss.com/microscopy/zh/products/software/zeiss-zen.html)
- [elastix](https://elastix.dev/).

### Steps to install python environment and libraries:

#### 1. Install Jupyter Notebook 7.x (or Python 3.x)
- `Jupyter Notebook 7.x` is available for download from the [official Jupyter website](https://jupyter.org/).
- `Python 3.x` is available for download from the [official Python website](https://www.python.org/).

#### 2. Update `pip` (optional)
It's a good practice to keep `pip` up to date. Open a terminal (or command prompt) and run the following command:

```bash
python -m pip install --upgrade pip
```

#### 3. Install Required Libraries
The required python libraries include: 
- `SimpleITK`
- `os`
- `cv2`
- `numpy`
- `shutil`
- `pandas`
- `PIL`
- `openpyxl`
- `itertools`
- `sklearn.metrics`
- `scipy.optimize`<br>

For example, to install `numpy`, execute the following command in your terminal or command prompt:

```bash
pip install numpy
```

#### 4. Verify Installation
After installation, you can verify that the libraries are installed correctly by running this Python script:

```python
import numpy as np
import cv2
import pandas as pd

print(f"numpy version: {np.__version__}")
print(f"cv2 version: {cv2.__version__}")
print(f"pandas version: {pd.__version__}")
```

This script will print the installed versions of the libraries. If you see the version numbers printed without any errors, the libraries were installed successfully.

### Run the code
- Execute the integrated Notebook from Step 1 to Step 7 in order.
- Test images are included in each subfolder.

## Step 1: Input images
Image obtained at time point 1 (0 min):<br>
- `T1_FAM`: FAM channel obtained at time point 1;
- `T1_Cy3`: Cy3 channel obtained at time point 1;
- `T1_Cy5`: Cy5 channel obtained at time point 1;
- `T1_align`: DAPI or merged FAM/Cy3/Cy5 channel obtained at time point 1;<br>

Image obtained at time point 2 (30 min):<br>
- `T2_FAM`: FAM channel obtained at time point 2;
- `T2_Cy3`: Cy3 channel obtained at time point 2;
- `T2_Cy5`: Cy5 channel obtained at time point 2;
- `T2_align`: DAPI or merged FAM/Cy3/Cy5 channel obtained at time point 2;<br>

Image obtained at time point 3 (60 min):<br>
- `T3_FAM`: FAM channel obtained at time point 3;
- `T3_Cy3`: Cy3 channel obtained at time point 3;
- `T3_Cy5`: Cy5 channel obtained at time point 3;
- `T3_align`: DAPI or merged FAM/Cy3/Cy5 channel obtained at time point 3;<br>

Note:
- Both DAPI channel (stained with cell nucleus) or merged FAM/Cy3/Cy5 channel can be utilized for image registration. It’s important to note that the optimal focal planes for DAPI, FAM, Cy3, and Cy5 may differ. Any subtle variations in the DAPI channel may inadvertently impact the image registration process for FAM, Cy3, and Cy5. In contrast, the merged channel of FAM, Cy3, and Cy5 channels provides a more accurate representation of the actual results from the FAM, Cy3, and Cy5 channels.
- To merge FAM, Cy3 and Cy5 channels, the raw image should be used in 8-bit or 16-bit grayscale mode. This standardization step is essential because different colors, when transformed to grayscale, can produce varying levels of brightness and contrast, potentially leading to misinterpretation of the relative intensities across channels.
- All images are captured from the same regions under identical microscopic parameters.

## Step 2: Image registration
This section aims to align fluorescence images obatined at different time points, which contains two main steps:
- `Step 1`: rigidly register the DAPI or merged FAM/Cy3/Cy5 images obtained at time point 2 and time point 3 with the one obtained at time point 1, and record the transformation parameters during the registration process.
- `Step 2`: apply the transformation parameters to other channels (FAM, Cy3, Cy5).<br>

To run tihs section:
- This Python code uses the `SimpleITK` library to call the `elastix` and `transformix` tools for rigid registration. Before running the code, please ensure that `elastix` and `transformix` are correctly installed, and the directories containing their executable files are added to the system’s environment variables so that the code can call them properly. In addition, you need to install the following modules in the notebook environment using the following commands:
```bash
pip install SimpleITK
pip install SimpleITK-SimpleElastix
```
Input image type: This code supports `8-bit TIFF` or `16-bit TIFF`.<br>
Output image type: The output format is consistent with the input format.<br>

- One can also use external software (provided in the '2_image_registration' folder) to achieve the same goal as this section.

In [8]:
import SimpleITK as sitk
import os
import cv2
import numpy as np

def rigid_registration(fixed_image, moving_image):
    """
    Perform rigid registration between fixed and moving images.
    :param fixed_image: Fixed image for registration.
    :param moving_image: Moving image to be registered to the fixed image.
    :return: Transformed moving image and the transformation parameter map.
    """
    elastix_image_filter = sitk.ElastixImageFilter()
    elastix_image_filter.SetFixedImage(fixed_image)
    elastix_image_filter.SetMovingImage(moving_image)

    # Get the default rigid registration parameter map
    parameter_map = sitk.GetDefaultParameterMap("rigid")
    elastix_image_filter.SetParameterMap(parameter_map)

    # Execute the registration
    elastix_image_filter.Execute()

    # Get the result image and the transformation parameter map
    result_image = elastix_image_filter.GetResultImage()
    transform_map = elastix_image_filter.GetTransformParameterMap()

    return result_image, transform_map


def apply_transform(moving_image, transform_map):
    """
    Apply the transformation to the moving image using the given parameter map.
    :param moving_image: Moving image to be transformed.
    :param transform_map: Transformation parameter map.
    :return: Transformed moving image.
    """
    transformix_image_filter = sitk.TransformixImageFilter()
    transformix_image_filter.SetMovingImage(moving_image)
    transformix_image_filter.SetTransformParameterMap(transform_map)
    transformix_image_filter.Execute()
    return transformix_image_filter.GetResultImage()


def get_image_type(image_path):
    """
    Get the pixel type of the image
    :param image_path: Path to the image
    :return: Pixel type string
    """
    image = sitk.ReadImage(image_path)
    pixel_id = image.GetPixelID()
    
    if pixel_id == sitk.sitkUInt8:
        return sitk.sitkUInt8
    elif pixel_id == sitk.sitkUInt16:
        return sitk.sitkUInt16
    else:
        raise ValueError(f"Unsupported pixel type for image: {image_path}")


# Input image names
image_names = [
    "T1_FAM", "T1_Cy3", "T1_Cy5", "T1_merge",
    "T2_FAM", "T2_Cy3", "T2_Cy5", "T2_merge",
    "T3_FAM", "T3_Cy3", "T3_Cy5", "T3_merge"
]

# Directory where the input images are stored
input_dir = "D://Customized algorithm for DySpec//1_input_image"
# Directory where the output images will be saved
output_dir = "D://Customized algorithm for DySpec//2_image_registration"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Read the fixed image (T1_merge)
fixed_image_name = "T1_merge"
fixed_image_path = os.path.join(input_dir, f"{fixed_image_name}.tif")
fixed_image_type = get_image_type(fixed_image_path)
fixed_image = sitk.ReadImage(fixed_image_path, fixed_image_type)

# Dictionary to store transformation parameter maps
transformation_maps = {}

# First align T2_merge and T3_merge to T1_merge
for time_point in [2, 3]:
    merge_image_name = f"T{time_point}_merge"
    merge_image_path = os.path.join(input_dir, f"{merge_image_name}.tif")
    
    # Read image with original type
    image_type = get_image_type(merge_image_path)
    merge_image = sitk.ReadImage(merge_image_path, image_type)
    
    # Perform rigid registration
    aligned_merge_image, transform_map = rigid_registration(fixed_image, merge_image)
    
    # Convert back to original type
    aligned_merge_image = sitk.Cast(aligned_merge_image, image_type)
    
    # Save the aligned merge image
    aligned_output_path = os.path.join(output_dir, f"aligned_{merge_image_name}.tif")
    sitk.WriteImage(aligned_merge_image, aligned_output_path)
    
    # Store the transformation map for later use
    transformation_maps[time_point] = transform_map

# Apply transformations to FAM, Cy3, Cy5 channels using corresponding merge image transforms
for time_point in [2, 3]:
    # Get the transformation map from the corresponding merge image
    transform_map = transformation_maps[time_point]
    
    # Apply the transformation to FAM, Cy3, Cy5 channels
    for channel in ["FAM", "Cy3", "Cy5"]:
        channel_image_name = f"T{time_point}_{channel}"
        channel_image_path = os.path.join(input_dir, f"{channel_image_name}.tif")
        
        # Read image with original type
        image_type = get_image_type(channel_image_path)
        channel_image = sitk.ReadImage(channel_image_path, image_type)
        
        # Apply the transformation
        aligned_channel_image = apply_transform(channel_image, transform_map)
        
        # Convert back to original type
        aligned_channel_image = sitk.Cast(aligned_channel_image, image_type)
        
        # Save the aligned channel image
        aligned_output_path = os.path.join(output_dir, f"aligned_{channel_image_name}.tif")
        sitk.WriteImage(aligned_channel_image, aligned_output_path)

print("Registration and transformation completed.")

Registration and transformation completed.


- It is recommended to input images that exhibit visible consistency to the unaided eye. Significant discrepancies between matched images may lead to alignment failure. Below, you'll find supplementary code to assess whether portions of the images can be successfully processed. If you encounter a warning stating "Failed to align XXX. Exiting.", please revise your input images accordingly.

In [2]:
import cv2
import numpy as np

def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}. Check the file path and integrity.")
    else:
        # Optionally resize the image to reduce memory usage
        print(f"Successfully loaded image from {image_path} with shape {image.shape}")
    return image

def align_images(base_image, target_image):
    if base_image is None or target_image is None:
        print("Error: One of the images is not loaded.")
        return None, None

    try:
        # Convert images to grayscale
        base_gray = cv2.cvtColor(base_image, cv2.COLOR_BGR2GRAY)
        target_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)
        
        # Initialize SIFT detector
        sift = cv2.SIFT_create()

        # Find the keypoints and descriptors with SIFT
        kp1, des1 = sift.detectAndCompute(base_gray, None)
        kp2, des2 = sift.detectAndCompute(target_gray, None)

        # Check if descriptors are found
        if des1 is None or des2 is None:
            print("Error: Descriptors not found in one of the images.")
            return None, None

        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)

        # Initialize FLANN matcher
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        # Match descriptors using KNN
        matches = flann.knnMatch(des1, des2, k=2)

        # Store good matches as per Lowe's ratio test.
        good_matches = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)

        if len(good_matches) > 4:
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

            # Calculate Homography
            H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
            # Warp target image to align with the base image
            aligned_image = cv2.warpPerspective(target_image, H, (base_image.shape[1], base_image.shape[0]))
            return aligned_image, H
        else:
            print("Error: Not enough matches found.")
            return None, None
    except cv2.error as e:
        print(f"OpenCV error: {e}")
        return None, None

# Load images
DAPI0 = load_image(r'D://Customized algorithm for DySpec//1_input_image//0_DAPI0.tif')
DAPI = load_image(r'D://Customized algorithm for DySpec//1_input_image//0_DAPI.tif')

# Align DAPI to DAPI0 if loaded successfully
if DAPI0 is not None and DAPI is not None:
    aligned_DAPI, H2 = align_images(DAPI0, DAPI)
    if aligned_DAPI is None:
        print("Failed to align DAPI. Exiting.")
        exit()

# Load channels for DAPI (CY5, CY3, FAM)
CY5 = load_image(r'D://Customized algorithm for DySpec//1_input_image//0_CY5.tif')
CY3 = load_image(r'D://Customized algorithm for DySpec//1_input_image//0_CY3.tif')
FAM = load_image(r'D://Customized algorithm for DySpec//1_input_image//0_FAM.tif')

if CY5 is not None and CY3 is not None and FAM is not None:
    # Apply the same transformation to channels
    aligned_CY5 = cv2.warpPerspective(CY5, H2, (DAPI0.shape[1], DAPI0.shape[0]))
    aligned_CY3 = cv2.warpPerspective(CY3, H2, (DAPI0.shape[1], DAPI0.shape[0]))
    aligned_FAM = cv2.warpPerspective(FAM, H2, (DAPI0.shape[1], DAPI0.shape[0]))
    print("Aligned channels for DAPI successfully.")
else:
    print("Failed to load one or more channels for DAPI. Exiting.")

# Save aligned images
if aligned_DAPI is not None:
    cv2.imwrite('D://Customized algorithm for DySpec//2_image_registration//0_aligned_DAPI.tif', aligned_DAPI)
    print("Aligned DAPI saved as 'aligned_DAPI.tif'.")

if aligned_CY5 is not None:
    cv2.imwrite('D://Customized algorithm for DySpec//2_image_registration//0_aligned_CY5.tif', aligned_CY5)
    print("Aligned CY5 saved as 'aligned_CY5.tif'.")

if aligned_CY3 is not None:
    cv2.imwrite('D://Customized algorithm for DySpec//2_image_registration//0_aligned_CY3.tif', aligned_CY3)
    print("Aligned CY3 saved as 'aligned_CY3.tif'.")

if aligned_FAM is not None:
    cv2.imwrite('D://Customized algorithm for DySpec//2_image_registration//0_aligned_FAM.tif', aligned_FAM)
    print("Aligned FAM saved as 'aligned_FAM.tif'.")

Successfully loaded image from D://Customized algorithm for DySpec//1_input_image//0_DAPI0.tif with shape (1024, 1024, 3)
Successfully loaded image from D://Customized algorithm for DySpec//1_input_image//0_DAPI.tif with shape (1024, 1024, 3)
Successfully loaded image from D://Customized algorithm for DySpec//1_input_image//0_CY5.tif with shape (1024, 1024, 3)
Successfully loaded image from D://Customized algorithm for DySpec//1_input_image//0_CY3.tif with shape (1024, 1024, 3)
Successfully loaded image from D://Customized algorithm for DySpec//1_input_image//0_FAM.tif with shape (1024, 1024, 3)
Aligned channels for DAPI successfully.
Aligned DAPI saved as 'aligned_DAPI.tif'.
Aligned CY5 saved as 'aligned_CY5.tif'.
Aligned CY3 saved as 'aligned_CY3.tif'.
Aligned FAM saved as 'aligned_FAM.tif'.


## Step 3: Image trim
- This step aims to trim the aligned images to a uniform size.
- Image trim for DAPI channels is unnecessary, as DAPI is displayed in the final merged images.
- The following Java code is executed within [Fiji](https://imagej.net/software/fiji/downloads) to accomplish this task:

In [ ]:
# To generate a customized code: Open Fiji → Plugins → Macro → Record
# To run the code: Open Fiji → Process → Batch → Macro
# Input folder: D:///Customized algorithm for DySpec//3_image_trim//Fiji_input
# Output folder: D:///Customized algorithm for DySpec//3_image_trim//Fiji_output
//setTool("rectangle");
makeRectangle(60, 32, 900, 900);
# To choose an appropriate size to crop: Open Fiji → open one of the image → choose Rectangle → the size will be shown in the software: x=4.18 (129), y=5.11 (315), w=0.50 (1024), h=0.46 (1024);
# The cropped dimensions will be: 1024px* 1024px, as illustrated by the values w=0.50 (1024), h=0.46 (1024);
run("Crop");

## Step 4: Image Subtraction
This algorithm based on the theory that when two images are merged, the fluorescence intensity of the combined image equals the sum of the intensities of the individual images (Image1+2 = Image1 + Image2). Conversely, by subtracting one image (Image1) from the merged image (Image1+2), the other image (Image2) can be resolved. 

- Decoding β-Tubulin from DySpec images

In [1]:
import cv2
import numpy as np

# Import images
ImageC1 = cv2.imread('D://Customized algorithm for DySpec//4_image_subtraction//T3_FAM.tif', cv2.IMREAD_GRAYSCALE)
ImageC2 = cv2.imread('D://Customized algorithm for DySpec//4_image_subtraction//T1_Cy3.tif', cv2.IMREAD_GRAYSCALE)

# Get the size of each image
height, width = ImageC1.shape

# Create an empty matrix to store the new gray value
new_gray_values = np.zeros((height, width), dtype=np.uint8)

# Traverse each pixel coordinate
for i in range(height):
    for j in range(width):
        # Calculate the new gray value according to the pre-set color barcode (the following is just an example for SV2)
        if ImageC1[i, j] > ImageC2[i, j]:
            new_gray_values[i, j] = ImageC1[i, j] - ImageC2[i, j]
        else:
            new_gray_values[i, j] = 0
# Convert the new grayscale value matrix to an image
cv2.imwrite('D://Customized algorithm for DySpec//4_image_subtraction//01_Tubulin.tif', new_gray_values)

True

- Decoding Vimentin from DySpec images

In [4]:
import cv2
import numpy as np
import os
import shutil

def copy_and_rename_tif_file(directory):
    
    source_path = os.path.join(directory, "D://Customized algorithm for DySpec//4_image_subtraction//T1_Cy3.tif")
    target_path = os.path.join(directory, "D://Customized algorithm for DySpec//4_image_subtraction//02_Vimentin.tif")
    
    try:
        # Check if the source file exists
        if not os.path.exists(source_path):
            print(f"Error: Source file {source_path} does not exist")
            return False
        
        # Check if the target file already exists
        if os.path.exists(target_path):
            print(f"Error: Target file {target_path} already exists")
            return False
        
        # Copy the file (preserves metadata)
        shutil.copy2(source_path, target_path)
        print(True)
        return True
        
    except Exception as e:
        print(f"Error: An exception occurred during copy operation: {e}")
        return False

if __name__ == "__main__":
    # Set the directory path directly in the code
    directory_path = "/path/to/your/tif/files"  # Replace with your actual path
    copy_and_rename_tif_file(directory_path)

True


- Decoding PKC-α from DySpec images

In [7]:
import cv2
import numpy as np
import os
import shutil

def copy_and_rename_tif_file(directory):
    
    source_path = os.path.join(directory, "D://Customized algorithm for DySpec//4_image_subtraction//T2_FAM.tif")
    target_path = os.path.join(directory, "D://Customized algorithm for DySpec//4_image_subtraction//03_PKC.tif")
    
    try:
        # Check if the source file exists
        if not os.path.exists(source_path):
            print(f"Error: Source file {source_path} does not exist")
            return False
        
        # Check if the target file already exists
        if os.path.exists(target_path):
            print(f"Error: Target file {target_path} already exists")
            return False
        
        # Copy the file (preserves metadata)
        shutil.copy2(source_path, target_path)
        print(True)
        return True
        
    except Exception as e:
        print(f"Error: An exception occurred during copy operation: {e}")
        return False

if __name__ == "__main__":
    # Set the directory path directly in the code
    directory_path = "/path/to/your/tif/files"  # Replace with your actual path
    copy_and_rename_tif_file(directory_path)

True


- Decoding Chx10 from DySpec images

In [10]:
import cv2
import numpy as np
import os
import shutil

def copy_and_rename_tif_file(directory):
  
    source_path = os.path.join(directory, "D://Customized algorithm for DySpec//4_image_subtraction//T3_Cy5.tif")
    target_path = os.path.join(directory, "D://Customized algorithm for DySpec//4_image_subtraction//04_Chx10.tif")
    
    try:
        # Check if the source file exists
        if not os.path.exists(source_path):
            print(f"Error: Source file {source_path} does not exist")
            return False
        
        # Check if the target file already exists
        if os.path.exists(target_path):
            print(f"Error: Target file {target_path} already exists")
            return False
        
        # Copy the file (preserves metadata)
        shutil.copy2(source_path, target_path)
        print(True)
        return True
        
    except Exception as e:
        print(f"Error: An exception occurred during copy operation: {e}")
        return False

if __name__ == "__main__":
    # Set the directory path directly in the code
    directory_path = "/path/to/your/tif/files"  # Replace with your actual path
    copy_and_rename_tif_file(directory_path)

True


- Decoding SV2 from DySpec images

In [13]:
import cv2
import numpy as np

# Import images
ImageC1 = cv2.imread('D://Customized algorithm for DySpec//4_image_subtraction//T1_FAM.tif', cv2.IMREAD_GRAYSCALE)
ImageC2 = cv2.imread('D://Customized algorithm for DySpec//4_image_subtraction//T2_FAM.tif', cv2.IMREAD_GRAYSCALE)

# Get the size of each image
height, width = ImageC1.shape

# Create an empty matrix to store the new gray value
new_gray_values = np.zeros((height, width), dtype=np.uint8)

# Traverse each pixel coordinate
for i in range(height):
    for j in range(width):
        if ImageC1[i, j] > ImageC2[i, j]:
            new_gray_values[i, j] = ImageC1[i, j] - ImageC2[i, j]
        else:
            new_gray_values[i, j] = 0
# Convert the new grayscale value matrix to an image
cv2.imwrite('D://Customized algorithm for DySpec//4_image_subtraction//05_SV2.tif', new_gray_values)

True

- Decoding Synapsin from DySpec images

In [14]:
import cv2
import numpy as np

# Import images
ImageC1 = cv2.imread('D://Customized algorithm for DySpec//4_image_subtraction//T2_Cy3.tif', cv2.IMREAD_GRAYSCALE)
ImageC2 = cv2.imread('D://Customized algorithm for DySpec//4_image_subtraction//T1_Cy3.tif', cv2.IMREAD_GRAYSCALE)

# Get the size of each image
height, width = ImageC1.shape

# Create an empty matrix to store the new gray value
new_gray_values = np.zeros((height, width), dtype=np.uint8)

# Traverse each pixel coordinate
for i in range(height):
    for j in range(width):
        # Calculate the new gray value according to the pre-set color barcode (the following is just an example for SV2)
        if ImageC1[i, j] > ImageC2[i, j]:
            new_gray_values[i, j] = ImageC1[i, j] - ImageC2[i, j]
        else:
            new_gray_values[i, j] = 0
# Convert the new grayscale value matrix to an image
cv2.imwrite('D://Customized algorithm for DySpec//4_image_subtraction//06_Synapsin.tif', new_gray_values)

True

## Step 5: Linear unmixing

For each pixel in the DySpec image, the color-changing information over different time points is stored in a one-dimensional array_m and displayed as follows: 
```
array_m = np.array([T1_FAM, T1_Cy3, T1_Cy5, T2_FAM, T2_Cy3, T2_Cy5, T3_FAM, T3_Cy3, T3_Cy5])
```

- `T1_FAM`: FAM channel obtained at time point 1 (0 min in this study)
- `T1_Cy3`: Cy3 channel obtained at time point 1 (0 min in this study)
- `T1_Cy5`: Cy5 channel obtained at time point 1 (0 min in this study)
- `T2_FAM`: FAM channel obtained at time point 2 (30 min in this study)
- `T2_Cy3`: Cy3 channel obtained at time point 2 (30 min in this study)
- `T2_Cy5`: Cy5 channel obtained at time point 2 (30 min in this study)
- `T3_FAM`: FAM channel obtained at time point 3 (60 min in this study)
- `T3_Cy3`: Cy3 channel obtained at time point 3 (60 min in this study)
- `T3_Cy5`: Cy5 channel obtained at time point 3 (60 min in this study)

The color-changing information for each pixel is stored in a worksheet, where the cell coordinates correspond to the pixel coordinates in the image.<br>

The fundamental concept of linear unmixing is based on the following formula:

 `array_m = C1*array_1 + C2*array_2 + …… + Cn*array_n`

- `array_m`：Color-changing information of each pixel.
- `array_1` ~ `array_n`: Spectrum-transforming fluorescent barcode of each protein.
- `contribution factor (C)`: Contribution of array_n in making up array_m, which also corresponds to the relative intensity of the recontrasted image.
- The mismatch score, represented by the Mean Squared Error (MSE), can be fine-tuned to achieve optimal results. In our study, a threshold of MSE < 20 is applied, and values exceeding this threshold are deemed unreliable and excluded from analysis (i.e., set to zero).

### 5.1 Extra color-changing information from raw images to one-dimensional arrays

In [1]:
import os
import pandas as pd
from PIL import Image
from openpyxl import Workbook

# image file path
image_files = [
    'D://Customized algorithm for DySpec//5_linear_unmixing//T1_FAM.tif',
    'D://Customized algorithm for DySpec//5_linear_unmixing//T1_Cy3.tif',
    'D://Customized algorithm for DySpec//5_linear_unmixing//T1_Cy5.tif',
    'D://Customized algorithm for DySpec//5_linear_unmixing//T2_FAM.tif',
    'D://Customized algorithm for DySpec//5_linear_unmixing//T2_Cy3.tif',
    'D://Customized algorithm for DySpec//5_linear_unmixing//T2_Cy5.tif',
    'D://Customized algorithm for DySpec//5_linear_unmixing//T3_FAM.tif',
    'D://Customized algorithm for DySpec//5_linear_unmixing//T3_Cy3.tif',
    'D://Customized algorithm for DySpec//5_linear_unmixing//T3_Cy5.tif'
]

# create a new workbook
wb = Workbook()
ws = wb.active
ws.title = "Gray Values"

# create a dictionary to store the grayscale values of coordinates
gray_value_dict = {}

# iterate through each image file
for image_file in image_files:
    with Image.open(image_file) as img:
        
        # get image size
        width, height = img.size
        
        # extract the grayscale value of each pixel
        for y in range(height):
            for x in range(width):
                value = img.getpixel((x, y))
                
                # use the coordinates (x, y) as the key in the dictionary
                if (x, y) not in gray_value_dict:
                    gray_value_dict[(x, y)] = []
                gray_value_dict[(x, y)].append(value)

# write the grayscale values to the worksheet
for (x, y), values in gray_value_dict.items():
    # Combine grayscale values with the same coordinates into a one-dimensional array
    combined_values = ', '.join(map(str, values))
    ws.cell(row=y + 1, column=x + 1, value=combined_values)

# write the grayscale values to the worksheet
output_file = 'D://Customized algorithm for DySpec//5_linear_unmixing//raw_color_changing_information.xlsx'
wb.save(output_file)

print(f"grayscale values have been exported to {output_file}")

grayscale values have been exported to D://Customized algorithm for DySpec//5_linear_unmixing//raw_color_changing_information.xlsx


### 5.2 Linear unmixing of six retinal proteins

In [4]:
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.metrics import mean_squared_error
from scipy.optimize import nnls
import openpyxl

# Read the color-changing information of each pixel
input_df = pd.read_excel('D://Customized algorithm for DySpec//5_linear_unmixing//raw_color_changing_information.xlsx', header=None)

# Spectrum-transforming fluorescent barcodes assigned to each protein
arrays = [
  np.array([0, 0, 1, 0, 0, 1, 1, 0, 0]), #1_β-Tubulin
  np.array([0, 1, 0, 0, 1, 0, 1, 0, 0]), #2_Vimentin
  np.array([1, 0, 0, 1, 0, 0, 0, 1, 0]), #3_PKC-α
  np.array([0, 0, 1, 0, 0, 1, 0, 0, 1]), #4_Chx10
  np.array([1, 0, 0, 0, 0, 1, 0, 1, 0]), #5_SV2
  np.array([0, 0, 1, 0, 1, 0, 0, 1, 0]), #6_Synapsin
]

def find_combinations(array_m, arrays, max_arrays=6):
    best_combination = None
    best_constants = None
    best_score = float('inf')

    for n in range(1, max_arrays + 1):
        for combo in combinations(enumerate(arrays), n):
            indices, selected_arrays = zip(*combo)
            
            A = np.array(selected_arrays).T
            b = array_m
            try:
                constants, _ = nnls(A, b)
                predicted = np.dot(A, constants)
                score = mean_squared_error(array_m, predicted)
                
                if score < best_score:
                    best_score = score
                    best_combination = indices
                    best_constants = constants
            except ValueError:
                continue

    return best_combination, best_constants, best_score

# Creat a new Excel file to store results
workbook = openpyxl.Workbook()

# Define the sheet with array and protein name
sheet_names = ['1_Tubulin', '2_Vimentin', '3_PKC', '4_Chx10', '5_SV2', '6_Synapsin', 'Mismatch Score']

# Create an empty DataFrame for each sheet and fill it with 0.0
results = {name: pd.DataFrame(0.0, index=input_df.index, columns=input_df.columns, dtype=float) for name in sheet_names}

# Calculate the optimal combination for each cell
for row in range(input_df.shape[0]):
    for col in range(input_df.shape[1]):
        cell_value = input_df.iloc[row, col]
        if pd.notna(cell_value):  # Check if the cell is empty
            array_m = np.array([float(x) for x in str(cell_value).split(',')])  # Convert the cell value to an array
            if len(array_m) == len(arrays[0]):  # Ensure length matches
                combination, constants, score = find_combinations(array_m, arrays)
                
                if score <= 20:
                    for i, const in zip(combination, constants):
                        results[sheet_names[i]].iloc[row, col] = const
                
                results['Mismatch Score'].iloc[row, col] = score
            else:
                print(f"Warning: Mismatch in array length at row {row+1}, column {col+1}")

# Write the results to Excel file
with pd.ExcelWriter('D://Customized algorithm for DySpec//5_linear_unmixing//contribution_factor.xlsx', engine='openpyxl') as writer:
    for sheet_name, df in results.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

print("Results have been written to contribution_factor.xlsx")

Results have been written to contribution_factor.xlsx


### 5.3 Linear unmixing of 20 breast cancer proteins

In [2]:
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.metrics import mean_squared_error
from scipy.optimize import nnls
import openpyxl

# Read the color-changing information of each pixel
input_df = pd.read_excel('D://Customized algorithm for DySpec//5_linear_unmixing//raw_color_changing_information.xlsx', header=None)

# Spectrum-transforming fluorescent barcodes assigned to each protein
arrays = [
  np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]), #1_CD3
  np.array([1, 0, 0, 0, 0, 1, 0, 1, 0]), #2_CD4
  np.array([0, 1, 0, 0, 0, 1, 1, 0, 0]), #3_CD8a
  np.array([0, 1, 0, 1, 0, 0, 0, 0, 1]), #4_CD19
  np.array([0, 0, 1, 0, 1, 0, 1, 0, 0]), #5_CD20
  np.array([0, 0, 1, 1, 0, 0, 0, 1, 0]), #6_CD45RA
  np.array([1, 0, 0, 0, 1, 0, 1, 0, 0]), #7_CD45RO
  np.array([1, 0, 0, 0, 0, 1, 1, 0, 0]), #8_CD56
  np.array([0, 1, 0, 1, 0, 0, 0, 1, 0]), #9_CD68
  np.array([0, 1, 0, 0, 0, 1, 0, 1, 0]), #10_CD80
  np.array([0, 0, 1, 1, 0, 0, 0, 0, 1]), #11_CD163
  np.array([0, 0, 1, 0, 1, 0, 0, 0, 1]), #12_Foxp3
  np.array([0, 1, 0, 1, 0, 0, 1, 0, 0]), #13_GZMB
  np.array([1, 0, 0, 0, 0, 1, 0, 0, 1]), #14_MPO
  np.array([1, 0, 0, 0, 1, 0, 0, 1, 0]), #15_HER2
  np.array([0, 1, 0, 0, 0, 1, 0, 0, 1]), #16_CK19
  np.array([0, 0, 1, 1, 0, 0, 1, 0, 0]), #17_PIP
  np.array([0, 0, 1, 0, 1, 0, 0, 1, 0]), #18_α-SMA
  np.array([1, 0, 0, 1, 0, 0, 0, 1, 0]), #19_CD31
  np.array([1, 0, 0, 1, 0, 0, 0, 0, 1]), #20_Ki-67
]

def find_combinations(array_m, arrays, max_arrays=3):
    best_combination = None
    best_constants = None
    best_score = float('inf')

    for n in range(1, max_arrays + 1):
        for combo in combinations(enumerate(arrays), n):
            indices, selected_arrays = zip(*combo)
            
            A = np.array(selected_arrays).T
            b = array_m
            try:
                constants, _ = nnls(A, b)
                predicted = np.dot(A, constants)
                score = mean_squared_error(array_m, predicted)
                
                if score < best_score:
                    best_score = score
                    best_combination = indices
                    best_constants = constants
            except ValueError:
                continue

    return best_combination, best_constants, best_score

# Creat a new Excel file to store results
workbook = openpyxl.Workbook()

# Define the sheet with array and protein name
sheet_names = ['1_CD3', '2_CD4', '3_CD8a', '4_CD19', '5_CD20', '6_CD45RA', '7_CD45RO', '8_CD56', '9_CD68', '10_CD80',
               '11_CD163', '12_Foxp3', '13_GZMB', '14_MPO', '15_HER2', '16_CK19', '17_PIP', '18_SMA', '19_CD31', '20_Ki67',
               'Mismatch Score']

# Create an empty DataFrame for each sheet and fill it with 0.0
results = {name: pd.DataFrame(0.0, index=input_df.index, columns=input_df.columns, dtype=float) for name in sheet_names}

# Calculate the optimal combination for each cell
for row in range(input_df.shape[0]):
    for col in range(input_df.shape[1]):
        cell_value = input_df.iloc[row, col]
        if pd.notna(cell_value):  # Check if the cell is empty
            array_m = np.array([float(x) for x in str(cell_value).split(',')])  # Convert the cell value to an array
            if len(array_m) == len(arrays[0]):  # Ensure length matches
                combination, constants, score = find_combinations(array_m, arrays)
                
                if score <= 20:
                    for i, const in zip(combination, constants):
                        results[sheet_names[i]].iloc[row, col] = const
                
                results['Mismatch Score'].iloc[row, col] = score
            else:
                print(f"Warning: Mismatch in array length at row {row+1}, column {col+1}")

# Write the results to Excel file
with pd.ExcelWriter('D://Customized algorithm for DySpec//5_linear_unmixing//contribution_factor.xlsx', engine='openpyxl') as writer:
    for sheet_name, df in results.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

print("Results have been written to contribution_factor.xlsx")

Results have been written to contribution_factor.xlsx


## Step 6: Image display

- To `show image`: The raw data containing the contribution factor was displayed as a grayscale image (.tif) using the following code.
- To assign the decoded image (grayscale) with `pseudo color`: Open zen 3.11 software → Dimensions → Channels → Change clor.
- To `merge images` of different proteins: Open zen 3.11 software → Processing → Method → Add channes.
- zen 3.11 software can be downloaded from the [official Zeiss software website](https://www.zeiss.com/microscopy/zh/products/software/zeiss-zen.html).

In [2]:
import openpyxl
from PIL import Image
import numpy as np

def excel_to_tiff(excel_path, output_dir):
    # Open the Excel file
    workbook = openpyxl.load_workbook(excel_path)

    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]

        # Get the dimensions of the Excel sheet
        max_row = sheet.max_row
        max_col = sheet.max_column

        # Create a numpy array to store grayscale values, with data type np.uint8 (8-bit)
        image_array = np.zeros((max_row, max_col), dtype=np.uint8)

        # Read values from the Excel sheet and populate the array
        for row in range(1, max_row + 1):
            for col in range(1, max_col + 1):
                cell_value = sheet.cell(row=row, column=col).value
                if cell_value is not None:
                    # Ensure the value is within the range of an 8-bit unsigned integer (0-255)
                    cell_value = max(0, min(int(cell_value), 255))
                    image_array[row - 1, col - 1] = cell_value

        # Create an image with 8-bit mode ('L' for grayscale)
        image = Image.fromarray(image_array, mode='L')

        # Construct the output file path, naming it after the sheet name
        tiff_path = f"{output_dir}/{sheet_name}.tif"

        # Save the image as a TIFF file
        image.save(tiff_path, format='TIFF')
        print(f"TIFF image has been saved to {tiff_path}")

# Specify the input Excel file path and output directory path
excel_file_path = r"D://Customized algorithm for DySpec//6_image_display//contribution_factor.xlsx"  # Replace with your Excel file path
output_directory = r"D://Customized algorithm for DySpec//6_image_display"  # Replace with the directory path where you want to save the TIFF files

# Call the function
excel_to_tiff(excel_file_path, output_directory)

TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/1_CD3.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/2_CD4.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/3_CD8a.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/4_CD19.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/5_CD20.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/6_CD45RA.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/7_CD45RO.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/8_CD56.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/9_CD68.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6_image_display/10_CD80.tif
TIFF image has been saved to D://Customized algorithm for DySpec//6

## Step 7: Pixel evaluation

- The F1 score ranges from 0 to 1, with 1 indicating perfect precision and recall, and 0 indicating the worst performance. F1 score was computed by comparing the decoded images and adjacent IF images. This comparison was conducted independently for each field of view and individual protein, using the following formula:<br>
`F1 Score=2TP/(2TP+FP+FN)`<br>
where TP represents true positive; FP represents false positive; and FN represents false negative.
- The calculations of the F1 score and Pearson correlation coefficient depend on the threshold value used to distinguish between negative and positive values, and are independent of whether the data is represented in 8-bit or 16-bit format.
- The DySpec-reconstructed images are compared with adjacent IF images at the pixel level, as DySpec and standard IF cannot be conducted on the same tissue. The comparative  results may be somewhat inferior than those observed directly from the same slide.

### 7.1 F1 score and person correction efficient with automatic threshold

- for paired comparison

In [11]:
import cv2
import numpy as np
from sklearn.metrics import f1_score

# Read 8-bit (or 16-bit) grayscale images
test_image = cv2.imread('D://Customized algorithm for DySpec//7_pixel_evaluation//reconstructed image.tif', cv2.IMREAD_GRAYSCALE)
groundtruth_image = cv2.imread('D://Customized algorithm for DySpec//7_pixel_evaluation//adjacent IF image.tif', cv2.IMREAD_GRAYSCALE)

# Check if images were successfully read
if test_image is None or groundtruth_image is None:
    print("Unable to read images, please check file paths and filenames.")
else:
    # Binarize using Otsu's method
    _, test_binary = cv2.threshold(test_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, groundtruth_binary = cv2.threshold(groundtruth_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Convert binary images to one-dimensional arrays
    test_flat = test_binary.flatten()
    groundtruth_flat = groundtruth_binary.flatten()

    # Convert arrays to 0 and 1 format
    test_flat = (test_flat / 255).astype(int)
    groundtruth_flat = (groundtruth_flat / 255).astype(int)

    # Calculate F1 score, setting zero_division parameter
    f1 = f1_score(groundtruth_flat, test_flat, zero_division=0)

    print(f"F1 score: {f1:.4f}")

    # Calculate Pearson correlation coefficient
    r = np.corrcoef(test_flat, groundtruth_flat)[0, 1]
    print(f"Pearson correlation coefficient (R): {r:.4f}")

F1 score: 0.9934
Pearson correlation coefficient (R): 0.9911


- for batch comparison

In [16]:
import cv2
import numpy as np
from sklearn.metrics import f1_score
import os

def calculate_metrics(test_path, gt_path):
    """Calculate F1 score and Pearson correlation coefficient for a single pair of images"""
    # Read grayscale images (supports 8-bit or 16-bit)
    test_image = cv2.imread(test_path, cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)
    gt_image = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE | cv2.IMREAD_ANYDEPTH)
    
    # Check if images were successfully read
    if test_image is None or gt_image is None:
        print(f"Warning: Unable to read images {test_path} or {gt_path}, skipping this pair")
        return None, None
    
    # Ensure image dimensions match
    if test_image.shape != gt_image.shape:
        print(f"Warning: Dimensions of {test_path} and {gt_path} do not match, skipping this pair")
        return None, None
    
    # Binarize images using Otsu's method
    _, test_binary = cv2.threshold(test_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, gt_binary = cv2.threshold(gt_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Convert to 1D arrays and normalize to 0-1 range
    test_flat = (test_binary.flatten() / 255).astype(int)
    gt_flat = (gt_binary.flatten() / 255).astype(int)
    
    # Calculate F1 score (handle zero-division cases)
    f1 = f1_score(gt_flat, test_flat, zero_division=0)
    
    # Calculate Pearson correlation coefficient
    r = np.corrcoef(test_flat, gt_flat)[0, 1]
    
    return f1, r

# Define image pairs (each pair contains prefixes of test image and ground truth image)
image_groups = [
    ("01_Tubulin", "1_Tubulin"),
    ("02_Vimentin", "2_Vimentin"),
    ("03_PKC", "3_PKC"),
    ("04_Chx10", "4_Chx10"),
    ("05_SV2", "5_SV2"),
    ("06_Synapsin", "6_Synapsin")
]

# Path to the folder containing images
folder_path = "D://Customized algorithm for DySpec//7_pixel_evaluation"

# Iterate through all pairs and calculate metrics
for test_prefix, gt_prefix in image_groups:
    # Construct image paths (assuming images are in tif format)
    test_path = os.path.join(folder_path, f"{test_prefix}.tif")
    gt_path = os.path.join(folder_path, f"{gt_prefix}.tif")
    
    print(f"\nProcessing: {test_prefix} and {gt_prefix}")
    f1, r = calculate_metrics(test_path, gt_path)
    
    if f1 is not None and r is not None:
        print(f"F1 score: {f1:.4f}")
        print(f"Pearson correlation coefficient (R): {r:.4f}")

print("\nAll pairs processed")


Processing: 01_Tubulin and 1_Tubulin
F1 score: 0.9481
Pearson correlation coefficient (R): 0.9449

Processing: 02_Vimentin and 2_Vimentin
F1 score: 0.4875
Pearson correlation coefficient (R): 0.4092

Processing: 03_PKC and 3_PKC
F1 score: 0.9584
Pearson correlation coefficient (R): 0.9487

Processing: 04_Chx10 and 4_Chx10
F1 score: 0.9689
Pearson correlation coefficient (R): 0.9589

Processing: 05_SV2 and 5_SV2
F1 score: 0.9401
Pearson correlation coefficient (R): 0.9370

Processing: 06_Synapsin and 6_Synapsin
F1 score: 0.9916
Pearson correlation coefficient (R): 0.9910

All pairs processed


### 7.2 F1 score and person correction efficient with manual threshold

In [12]:
import cv2
import numpy as np
from sklearn.metrics import f1_score

# Read 8-bit (or 16-bit) grayscale images
test_image = cv2.imread('D://Customized algorithm for DySpec//7_pixel_evaluation///reconstructed image.tif', cv2.IMREAD_GRAYSCALE)
groundtruth_image = cv2.imread('D://Customized algorithm for DySpec//7_pixel_evaluation//adjacent IF image.tif', cv2.IMREAD_GRAYSCALE)

# Check if images were successfully read
if test_image is None or groundtruth_image is None:
    print("Unable to read images, please check file paths and filenames.")
else:
    # Use a fixed threshold of 10 for binarization (This value can be adjusted based on actual conditions)
    _, test_binary = cv2.threshold(test_image, 1, 255, cv2.THRESH_BINARY)
    _, groundtruth_binary = cv2.threshold(groundtruth_image, 1, 255, cv2.THRESH_BINARY)

    # Convert binary images to one-dimensional arrays
    test_flat = test_binary.flatten()
    groundtruth_flat = groundtruth_binary.flatten()

    # Convert arrays to 0 and 1 format
    test_flat = (test_flat / 255).astype(int)
    groundtruth_flat = (groundtruth_flat / 255).astype(int)

    # Calculate F1 score, setting zero_division parameter
    f1 = f1_score(groundtruth_flat, test_flat, zero_division=0)

    print(f"F1 score: {f1:.4f}")

    # Calculate Pearson correlation coefficient
    r = np.corrcoef(test_flat, groundtruth_flat)[0, 1]
    print(f"Pearson correlation coefficient (R): {r:.4f}")

F1 score: 1.0000
Pearson correlation coefficient (R): 1.0000
